# Functional Closures and Decorators

Sometimes we may have "cross-cutting" concerns: functionality which is not related to the "core" functionality of a function:

- logging
- authentication
- locking
- entry of the function into a global registry

Python has a syntax that uses two features of the language (scoping and first-class functions) to enable such operations to be separated from the core function.

First, though, let's see if you can guess what the following code prints:

In [ ]:
a = 5
b = 12
c = 30
def foo():
    a = 10
    b = 15
    print(f'1 => a={a}, b={b}, c={c}')  # '1 => a={}, b={}'.format(a, b)   Py 3.5+
    def bar():
        a = 20
        print(f'2 => a={a}, b={b}, c={c}')
    bar()
    print(f'3 => a={a}, b={b}, c={c}')
foo()
print(f'4 => a={a}, b={b}, c={c}')


##  LEGB
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing (i.e. `nonlocal`)
  * G = global (i.e. `global`)
  * B = builtin (e.g., len() function)
  
```python
a = 'global scope'

def outer_func():
    b = 'local to outer_func()'
    def inner_func():
        c = 'local to inner_func()'
        print(b, 'enclosing/nonlocal scope')
        print(a, 'global scope')
    return inner_func
```
                
* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which "remembers" a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

## `global` and `nonlocal` keywords

Python scoping works as described above for *reading* values associated with names

When **assigning** values, assignment is local *unless*

- the value is declared as `global`, in which case it assigns globally
- the value is declared as `nonlocal`, in which case it assigns to the "nearest" enclosing scope

# Closures

In [ ]:
def make_adder(x):
    
    def adder(y):
        return x + y # Python uses LEGB to find 'x'
    
    return adder

In [ ]:
add39 = make_adder(x=39)
add39

In [ ]:
add39(y=10)

In [ ]:
add12 = make_adder(12)
add12(10)

In [ ]:
add39(10)

(This is for enrichment, you will likely not need the details of implementation in your 'real' code)

In [ ]:
add39.__closure__

In [ ]:
cell = add39.__closure__[0]

In [ ]:
cell.cell_contents

In [ ]:
x = make_adder(39)
x(10)

In [ ]:
add6 = make_adder(6)

In [ ]:
print(add6(10), add39(10))

In [ ]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
#type(add39)
repr(add39)

In [ ]:
repr(add6)

In [ ]:
from IPython.display import IFrame
IFrame("""
https://pythontutor.com/iframe-embed.html#code=def%20make_adder%28x%29%3A%0A%20%20%20%20%0A%20%20%20%20def%20adder%28y%29%3A%0A%20%20%20%20%20%20%20%20return%20x%20%2B%20y%20%23%20Python%20uses%20LEGB%20to%20find%20'x'%0A%20%20%20%20%0A%20%20%20%20return%20adder%0A%20%20%20%20%0Aadd39%20%3D%20make_adder%2839%29%0Aadd6%20%3D%20make_adder%286%29%0Aprint%28add39%2812%29%29%0Aprint%28add6%285%29%29&codeDivHeight=400&codeDivWidth=350&cumulative=true&curInstr=0&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false
""", width=800, height=500)

## Building function wrappers

* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [ ]:
def myprinter(x):
    print('The value of x is', x)

In [ ]:
myprinter(22)

In [ ]:
def logging(function):
    print('Wrapping {}'.format(function))
    def wrapper(*args, **kwargs):
        # print('Calling %r(%r, %r)' % (function, args, kwargs))
        print(f'Calling {function!r}({args!r}, {kwargs!r})')
        return function(*args, **kwargs)
    return wrapper

In [ ]:
logging_myprinter = logging(myprinter)

In [ ]:
logging_myprinter

In [ ]:
myprinter(42)

In [ ]:
logging_myprinter(42)

In [ ]:
logging_myprinter(5)

In [ ]:
def add2(x, y):
    return x+y

In [ ]:
logging_add2 = logging(add2)

In [ ]:
logging_add2(10, 12)

(again, for enrichment)

In [ ]:
logging_myprinter.__closure__[0].cell_contents

(back to the main idea...)

In [ ]:
myprinter = logging(myprinter)

In [ ]:
myprinter

In [ ]:
myprinter(5)

In [ ]:
myprinter(22)

In [ ]:
def myprinter(x):
    print('The value of x is', x)
myprinter = logging(myprinter)

In [ ]:
@logging
def myprinter(x):
    print('The value of x is', x)

In [ ]:
myprinter(22)

# Definition of 'function decorator'

> A function that takes another function as its only argument and returns a replacement for the function that was passed in

# Non-wrapping decorators

In [ ]:
# Flask-like routing

routes = {}  # map url => python function


def route(path):   # "decorator factory" (returns a decorator)
    print(f'Build route({path})')
#     print('Build route({path})'.format(path=path))
#     print('Build route(%s)' % path)
    def decorator(function):   # decorator (takes a single function as input)
        print(f'Call decorator({path})({function})')
        routes[path] = function
        # (no wrapping going on here)
        return function
    return decorator

In [ ]:
@route('/')
def get_home():
    return 'Welcome'
print('routes', routes)

In [ ]:
# route_home_deco = route('/')
# def get_home():
#     return 'Welcome'
# get_home = route_home_deco(get_home)

@route('/about')
def get_about():
    return 'About us'
print('routes', routes)

In [ ]:
@route('/error')
@route('/other_error')
def get_error():
    return '404'
print('routes', routes)

# route_error_decorator = route('/error')
# route_other_error_decorator = route('/other_error')
# def get_error():
#     return '404'
# get_error = route_other_error_decorator(get_error)
# get_error = route_error_decorator(get_error)

In [ ]:
routes

In [ ]:
get_home

In [ ]:
def run_request(routes, url):
    function_to_call = routes[url]
    return function_to_call()

In [ ]:
run_request(routes, '/')

In [ ]:
run_request(routes, '/about')

In [ ]:
run_request(routes, '/error')

Wrapping order matters...

```python
def with_api_client(func):
    def wrapper(*args, **kwargs):
        cli = get_client_function()
        return func(cli, *args, **kwargs)
    return wrapper


@app.route('/patient/<patientId>')
@with_api_client
def get_patients(cli, patientId):
    ...

@with_api_client
@app.route('/patient/<patientId>')
def get_patients(cli, patientId):
    ...
```

```python
@EXPR0  # (
@EXPR1  # (
def FUNC():
    ...
# ))
```
    
... means nothing more than ...

```python
_temp0 = EXPR0
_temp1 = EXPR1
def FUNC():
    ...
FUNC = _temp1(FUNC)
FUNC = _temp0(FUNC)
```

In [ ]:
def my_decorator_factory(a, b, c=5):   # decorator factory
    print('calling my_decorator_factory')
    def decorator(func):               # decorator
        print('Calling decorator')
        def wrapper(*args, **kwargs):  # wrapper
            print(f'Calling wrapper with {a}, {b}, {c} on {func}')
            return func(*args, **kwargs)
        return wrapper
    return decorator

@my_decorator_factory('foo', 'bar')
def somefunc(x, y):
    return x + y

In [ ]:
_deco = my_decorator_factory('foo', 'bar')
def somefunc(x, y):
    return x + y
somefunc = _deco(somefunc)

In [ ]:
somefunc(1, 2)

In [ ]:
somefunc(5, 10)

# Auth psuedo-code

```python
def authorize_or_403(perms):
    ...
    
def require_authorized(perms):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            authorize_or_403(perms)
            return func(*args, **kwargs)
        return wrapper
    return decorator

@route('/protected')
def get_some_protected_thing():
    authorize_or_403('Patient.read')
    ...

@route('/protected2')
@require_authorized('Patient.read')
def get_some_other_protected_thing():
    ...
```

# Preserving introspection

In [ ]:
def document_it(func):
    "This is my document_it docstring"
    # below is a nested, or inner function
    def wrapper(*args, **kwargs):
        "wrapper docstring"
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return wrapper

In [ ]:
@document_it
def addit(x, y):
    """I heard I should use docstrings so here ya go"""
    return x + y

In [ ]:
addit(1,2)

In [ ]:
help(addit)

In [ ]:
addit

In [ ]:
from functools import wraps

def document_it(func):
    """This is the decorator docstring"""
    # below is a nested, or inner function
    @wraps(func)
    def wrapper(*args, **kwargs):
        """This is the wrapper docstring"""
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return wrapper

In [ ]:
@document_it
def addit(x, y):
    """I heard I should use docstrings so here ya go"""
    return x + y

In [ ]:
addit(1,2)

In [ ]:
help(addit)

In [ ]:
addit

# Lab

Open the [Closures Lab][closures-lab]

[closures-lab]: ./closures-lab.ipynb
